In [3]:

import numpy as np
from itertools import product
import matplotlib.pyplot as plt

# Define the adjacency matrices
adjacency_matrices = [
    np.array([[-1, -1], [-1, -1]]),
    np.array([[-1, -1], [-1, 0]]),
    np.array([[-1, -1], [-1, 1]]),
    np.array([[-1, -1], [0, -1]]),
    np.array([[-1, -1], [0, 0]]),
    np.array([[-1, -1], [0, 1]]),
    np.array([[-1, -1], [1, -1]]),
    np.array([[-1, -1], [1, 0]]),
    np.array([[-1, -1], [1, 1]]),
    np.array([[-1, 0], [-1, 0]]),
    np.array([[-1, 0], [-1, 1]]),
    np.array([[-1, 0], [0, -1]]),
    np.array([[-1, 0], [0, 0]]),
    np.array([[-1, 0], [0, 1]]),
    np.array([[-1, 0], [1, -1]]),
    np.array([[-1, 0], [1, 0]]),
    np.array([[-1, 0], [1, 1]]),
    np.array([[-1, 1], [-1, 0]]),
    np.array([[-1, 1], [-1, 1]]),
    np.array([[-1, 1], [0, 0]]),
    np.array([[-1, 1], [0, 1]]),
    np.array([[-1, 1], [1, -1]]),
    np.array([[-1, 1], [1, 0]]),
    np.array([[-1, 1], [1, 1]]),
    np.array([[0, -1], [-1, 0]]),
    np.array([[0, -1], [-1, 1]]),
    np.array([[0, -1], [0, 0]]),
    np.array([[0, -1], [0, 1]]),
    np.array([[0, -1], [1, 0]]),
    np.array([[0, -1], [1, 1]]),
    np.array([[0, 0], [-1, 1]]),
    np.array([[0, 0], [0, 0]]),
    np.array([[0, 0], [0, 1]]),
    np.array([[0, 0], [1, 0]]),
    np.array([[0, 0], [1, 1]]),
    np.array([[0, 1], [-1, 1]]),
    np.array([[0, 1], [0, 1]]),
    np.array([[0, 1], [1, 0]]),
    np.array([[0, 1], [1, 1]]),
    np.array([[1, -1], [-1, 1]]),
    np.array([[1, -1], [0, 1]]),
    np.array([[1, -1], [1, 1]]),
    np.array([[1, 0], [0, 1]]),
    np.array([[1, 0], [1, 1]]),
    np.array([[1, 1], [1, 1]])
]

def signum(x):
    """Signum function: returns -1 if x < 0, 0 if x == 0, 1 if x > 0."""
    return np.where(x < 0, -1, np.where(x == 0, 0, 1))

def update_state_sync(state, matrix):
    """Synchronous update of the Boolean network state."""
    new_state = signum(np.dot(matrix, state))
    return new_state

def update_state_async(state, matrix, node_index):
    """Asynchronous update: update only one node specified by node_index."""
    new_state = state.copy()
    new_state[node_index] = signum(np.dot(matrix[node_index], state))
    return new_state

def get_boolean_function(matrix):
    """Compute the Boolean function for each node."""
    truth_table = []
    for x1, x2 in product([-1, 1], repeat=2):
        state = np.array([x1, x2])
        next_state = update_state_sync(state, matrix)
        # Convert to Boolean: -1 -> 0, 1 -> 1
        bool_input = (0 if x1 == -1 else 1, 0 if x2 == -1 else 1)
        bool_output = tuple(0 if x == -1 else 1 for x in next_state)
        truth_table.append((bool_input, bool_output))

    def truth_to_expression(outputs):
        """Convert truth table outputs to a Boolean expression."""
        if outputs == [0, 0, 0, 0]:
            return "0"
        if outputs == [1, 1, 1, 1]:
            return "1"
        if outputs == [0, 0, 0, 1]:
            return "x1 & x2"
        if outputs == [0, 0, 1, 0]:
            return "x1 & !x2"
        if outputs == [0, 0, 1, 1]:
            return "x1"
        if outputs == [0, 1, 0, 0]:
            return "!x1 & x2"
        if outputs == [0, 1, 0, 1]:
            return "x2"
        if outputs == [0, 1, 1, 0]:
            return "!(x1 & x2)"
        if outputs == [0, 1, 1, 1]:
            return "!x1 | x2"
        if outputs == [1, 0, 0, 0]:
            return "!x1 & !x2"
        if outputs == [1, 0, 0, 1]:
            return "!x1"
        if outputs == [1, 0, 1, 0]:
            return "x1 ^ x2"
        if outputs == [1, 0, 1, 1]:
            return "x1 | !x2"
        if outputs == [1, 1, 0, 0]:
            return "!x2"
        if outputs == [1, 1, 0, 1]:
            return "!x1 | !x2"
        if outputs == [1, 1, 1, 0]:
            return "x1 | x2"
        return None

    node1_outputs = [t[1][0] for t in truth_table]
    node2_outputs = [t[1][1] for t in truth_table]

    expr1 = truth_to_expression(node1_outputs)
    expr2 = truth_to_expression(node2_outputs)

    return expr1, expr2

def find_sync_attractors(matrix):
    """Find all steady states and cycles for a given adjacency matrix (synchronous)."""
    n = matrix.shape[0]
    attractors = []
    visited = set()

    for initial_state in product([-1, 1], repeat=n):
        initial_state = np.array(initial_state)
        state_tuple = tuple(initial_state)

        if state_tuple in visited:
            continue

        current_state = initial_state.copy()
        trajectory = [tuple(current_state)]

        while True:
            next_state = update_state_sync(current_state, matrix)
            next_state_tuple = tuple(next_state)

            if next_state_tuple in visited:
                if next_state_tuple in trajectory:
                    cycle_start = trajectory.index(next_state_tuple)
                    cycle = trajectory[cycle_start:]
                    if len(cycle) == 1:
                        attractors.append(("Steady State", [list(cycle[0])]))
                    else:
                        attractors.append(("Cycle", [list(s) for s in cycle]))
                break

            visited.add(next_state_tuple)
            trajectory.append(next_state_tuple)
            current_state = next_state

    num_steady = sum(1 for type_, _ in attractors if type_ == "Steady State")
    num_cycles = sum(1 for type_, _ in attractors if type_ == "Cycle")
    cycle_lengths = [len(states) for type_, states in attractors if type_ == "Cycle"]
    avg_cycle_length = sum(cycle_lengths) / len(cycle_lengths) if cycle_lengths else 0

    return attractors, num_steady, num_cycles, avg_cycle_length

def find_async_attractors(matrix):
    """Find all steady states and cycles with steps for a given adjacency matrix (asynchronous)."""
    n = matrix.shape[0]
    attractors = []
    visited = set()
    steps_by_initial = {}

    for initial_state in product([-1, 1], repeat=n):
        initial_state = np.array(initial_state)
        state_tuple = tuple(initial_state)

        if state_tuple in visited:
            steps_by_initial[state_tuple] = [("Skipped", "Already visited", [])]
            continue

        current_state = initial_state.copy()
        trajectory = [(tuple(current_state), -1)]
        steps = [(list(current_state), -1, list(current_state))]
        node_index = 0

        while True:
            next_state = update_state_async(current_state, matrix, node_index)
            next_state_tuple = tuple(next_state)

            steps.append((list(current_state), node_index, list(next_state)))

            is_steady = True
            for ni in range(n):
                if not np.array_equal(update_state_async(next_state, matrix, ni), next_state):
                    is_steady = False
                    break
            if is_steady:
                attractors.append(("Steady State", [list(next_state)]))
                visited.add(next_state_tuple)
                steps_by_initial[state_tuple] = steps
                break

            current_pair = (next_state_tuple, node_index)
            if current_pair in trajectory:
                cycle_start = trajectory.index(current_pair)
                cycle = [s for s, _ in trajectory[cycle_start:]]
                if len(cycle) > 1 or not np.array_equal(cycle[0], next_state):
                    attractors.append(("Cycle", [list(s) for s in cycle]))
                for s, _ in trajectory[cycle_start:]:
                    visited.add(s)
                steps_by_initial[state_tuple] = steps
                break

            visited.add(next_state_tuple)
            trajectory.append(current_pair)
            current_state = next_state
            node_index = (node_index + 1) % n

            if len(trajectory) > 100:
                steps_by_initial[state_tuple] = steps + [("Terminated", "Max steps reached", [])]
                break

    unique_attractors = []
    seen_states = set()
    for type_, states in attractors:
        states_frozen = frozenset(tuple(s) for s in states)
        if states_frozen not in seen_states:
            unique_attractors.append((type_, states))
            seen_states.add(states_frozen)

    num_steady = sum(1 for type_, _ in unique_attractors if type_ == "Steady State")
    num_cycles = sum(1 for type_, _ in unique_attractors if type_ == "Cycle")
    cycle_lengths = [len(states) for type_, states in unique_attractors if type_ == "Cycle"]
    avg_cycle_length = sum(cycle_lengths) / len(cycle_lengths) if cycle_lengths else 0

    return unique_attractors, num_steady, num_cycles, avg_cycle_length, steps_by_initial

def plot_attractor_metrics(sync_metrics, async_metrics):
    """Create a bar plot comparing synchronous and asynchronous attractor metrics."""
    matrix_indices = np.arange(1, len(sync_metrics) + 1)
    sync_steady = [m[0] for m in sync_metrics]
    sync_cycles = [m[1] for m in sync_metrics]
    sync_avg_cycle = [m[2] for m in sync_metrics]
    async_steady = [m[0] for m in async_metrics]
    async_cycles = [m[1] for m in async_metrics]
    async_avg_cycle = [m[2] for m in async_metrics]

    width = 0.15
    fig, ax = plt.subplots(figsize=(18, 6))

    ax.bar(matrix_indices - 2.5*width, sync_steady, width, label='Sync Steady States', color='blue')
    ax.bar(matrix_indices - 1.5*width, sync_cycles, width, label='Sync Cycles', color='cyan')
    ax.bar(matrix_indices - 0.5*width, sync_avg_cycle, width, label='Sync Avg Cycle Length', color='lightblue')
    ax.bar(matrix_indices + 0.5*width, async_steady, width, label='Async Steady States', color='red')
    ax.bar(matrix_indices + 1.5*width, async_cycles, width, label='Async Cycles', color='orange')
    ax.bar(matrix_indices + 2.5*width, async_avg_cycle, width, label='Async Avg Cycle Length', color='yellow')

    ax.set_xlabel('Matrix Index')
    ax.set_ylabel('Metric Value')
    ax.set_title('Synchronous vs. Asynchronous Attractor Metrics for 45 Matrices')
    ax.set_xticks(matrix_indices)
    ax.legend()
    ax.grid(True, axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('attractor_metrics_combined.png')
    plt.close()

def generate_latex_table(sync_metrics, async_metrics):
    """Generate a LaTeX table of synchronous and asynchronous attractor metrics."""
    latex_content = r"""
\documentclass{article}
\usepackage{booktabs}
\usepackage[utf8]{inputenc}
\usepackage[T1]{fontenc}
\usepackage{lmodern}
\usepackage{geometry}
\geometry{letterpaper, margin=1in}

\begin{document}

\title{Synchronous and Asynchronous Attractor Metrics for 45 Boolean Network Matrices}
\author{}
\date{\today}
\maketitle

\section{Introduction}
This table summarizes the attractor metrics for 45 2x2 adjacency matrices in a Boolean network, computed using both synchronous and asynchronous updates. Metrics include the number of steady states, number of cycles, and average cycle length for each update scheme.

\begin{table}[h]
\centering
\small
\setlength{\tabcolsep}{3pt}
\renewcommand{\arraystretch}{1.2}
\begin{tabular}{c c c c c c c}
\toprule
\textbf{Matrix} & \multicolumn{3}{c}{\textbf{Synchronous}} & \multicolumn{3}{c}{\textbf{Asynchronous}} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7}
& \textbf{Steady} & \textbf{Cycles} & \textbf{Avg. Cycle} & \textbf{Steady} & \textbf{Cycles} & \textbf{Avg. Cycle} \\
\midrule
"""
    for i, (sync_m, async_m) in enumerate(zip(sync_metrics, async_metrics), 1):
        sync_steady, sync_cycles, sync_avg_cycle = sync_m
        async_steady, async_cycles, async_avg_cycle = async_m
        latex_content += f"{i} & {sync_steady} & {sync_cycles} & {sync_avg_cycle:.2f} & {async_steady} & {async_cycles} & {async_avg_cycle:.2f} \\\\\n"

    latex_content += r"""
\bottomrule
\end{tabular}
\caption{Attractor metrics for 45 Boolean network matrices. Synchronous and asynchronous updates are compared, showing the number of steady states, cycles, and average cycle length (0 if no cycles).}
\label{tab:attractor_metrics_combined}
\end{table}

\end{document}
"""
    with open('attractor_metrics_combined.tex', 'w') as f:
        f.write(latex_content)

def main():
    """Analyze all matrices, compute Boolean functions, synchronous and asynchronous attractors, and generate outputs."""
    sync_metrics = []
    async_metrics = []
    all_steps = []
    all_sync_attractors = []
    all_async_attractors = []

    for i, matrix in enumerate(adjacency_matrices, 1):
        print(f"\nMatrix {i}:")
        print(matrix)

        # Boolean function
        expr1, expr2 = get_boolean_function(matrix)
        print("\nBoolean Function:")
        print(f"x1' = {expr1}")
        print(f"x2' = {expr2}")

        # Synchronous attractors
        sync_attractors, sync_num_steady, sync_num_cycles, sync_avg_cycle = find_sync_attractors(matrix)
        sync_metrics.append((sync_num_steady, sync_num_cycles, sync_avg_cycle))
        all_sync_attractors.append(sync_attractors)
        print("\nSynchronous Update Scheme:")
        if not sync_attractors:
            print("No attractors found.")
        else:
            for attractor_type, states in sync_attractors:
                print(f"{attractor_type}:")
                for state in states:
                    print(f"  {state}")

        # Asynchronous attractors
        async_attractors, async_num_steady, async_num_cycles, async_avg_cycle, steps_by_initial = find_async_attractors(matrix)
        async_metrics.append((async_num_steady, async_num_cycles, async_avg_cycle))
        all_steps.append(steps_by_initial)
        all_async_attractors.append(async_attractors)
        print("\nAsynchronous Update Scheme:")
        if not async_attractors:
            print("No attractors found.")
        else:
            print("Attractors:")
            for attractor_type, states in async_attractors:
                print(f"{attractor_type}:")
                for state in states:
                    print(f"  {state}")

        print("\nAsynchronous Steps to Attractors:")
        for initial_state in product([-1, 1], repeat=2):
            initial_state = tuple(initial_state)
            print(f"Initial State: {list(initial_state)}")
            steps = steps_by_initial.get(initial_state, [])
            if not steps or steps[0][0] == "Skipped":
                print("  Skipped: Already visited")
                continue
            for step_num, (current, node, next_state) in enumerate(steps[1:], 1):
                if isinstance(current, str):
                    print(f"  Step {step_num}: {current}")
                    break
                print(f"  Step {step_num}: {current} --(Node {node})--> {next_state}")
            final_state = steps[-1][2] if steps[-1][0] != "Terminated" else None
            if final_state:
                for type_, states in async_attractors:
                    if any(np.array_equal(final_state, s) for s in states):
                        print(f"  Reached {type_}: {states}")
                        break

    # Plot the metrics
    plot_attractor_metrics(sync_metrics, async_metrics)

    # Generate LaTeX table
    generate_latex_table(sync_metrics, async_metrics)

if __name__ == "__main__":
    main()



Matrix 1:
[[-1 -1]
 [-1 -1]]

Boolean Function:
x1' = x1 | x2
x2' = x1 | x2

Synchronous Update Scheme:
Cycle:
  [np.int64(1), np.int64(1)]
  [np.int64(-1), np.int64(-1)]
Steady State:
  [np.int64(0), np.int64(0)]

Asynchronous Update Scheme:
Attractors:
Cycle:
  [np.int64(1), np.int64(-1)]
  [np.int64(1), np.int64(0)]
  [np.int64(-1), np.int64(0)]
  [np.int64(-1), np.int64(1)]
  [np.int64(0), np.int64(1)]
  [np.int64(0), np.int64(-1)]

Asynchronous Steps to Attractors:
Initial State: [-1, -1]
  Step 1: [np.int64(-1), np.int64(-1)] --(Node 0)--> [np.int64(1), np.int64(-1)]
  Step 2: [np.int64(1), np.int64(-1)] --(Node 1)--> [np.int64(1), np.int64(0)]
  Step 3: [np.int64(1), np.int64(0)] --(Node 0)--> [np.int64(-1), np.int64(0)]
  Step 4: [np.int64(-1), np.int64(0)] --(Node 1)--> [np.int64(-1), np.int64(1)]
  Step 5: [np.int64(-1), np.int64(1)] --(Node 0)--> [np.int64(0), np.int64(1)]
  Step 6: [np.int64(0), np.int64(1)] --(Node 1)--> [np.int64(0), np.int64(-1)]
  Step 7: [np.int64(0),